<a href="https://colab.research.google.com/github/kokinakag20-create/Asolabor/blob/main/asolabor3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# マルチエージェントシミュレーションを使った人の動きのシミュレーション

* 東京都立大学 都市環境学部 地理環境学科 中山大地
* 2023年11月18日 東京都立大学 南大沢キャンパス
* このテキストのURL [https://github.com/bokutachi256/gisday2023](https://github.com/bokutachi256/gisday2023)
* Google ColaboratoryのURL [https://colab.research.google.com/](https://colab.research.google.com/)

## 改変履歴

* 2024年11月14日：mesaのバージョンを2.4.0に指定

# 使い方

## データの準備

1. Googleドライブに`gisday2023`フォルダを作成する．
2. `data`フォルダに加重コスト距離のラスタデータファイル（例：`CostDis.tif`），スタート地点のGeoJSONファイル（`start_points.geojson`），ゴール地点のGeoJSONファイル（`goal_points.geojson`），道路のラスタデータファイル（`road.tif`）を格納する．
   1. すべてのデータは同じ座標系であること．
   2. ラスタデータはすべて同じサイズであること．
   3. 道路（壁）ファイルは通れる部分が1以上，通れない部分が0のラスタファイル
   4. 加重コスト距離ファイルは複数でも可
   5. スタート地点のGeoJSONに必須の属性
      * `cost_ID`: どの加重コスト距離に従うかを示す属性
   6. ゴール地点のGeoJSONに必須の属性は特になし．

## 計算結果をQGISで図化する

* gpkgの読み込み
  1. GoogleDriveからgpkgファイルをダウンロードする．
  1. gpkgファイルをQGISで読み込む．
* 時系列プロパティの設定
  1. gpkgレイヤーのプロパティを表示し，左側のボタンから「時系列」を選ぶ．
  1. 「動的時系列コントロール」にチェックを入れる．
  1. 「設定」を「Date/Time型の単一フィールド」，「上限」を「始点を含み終点を除く（デフォルト）」，「フィールド」を「Time」，「継続時間」を5秒にする．「地物を累積表示」にはチェックを入れない．
* 時系列表示の設定
  1. 「ビュー」メニューの「パネル」から「時系列コントローラーパネル」を表示する．
  1. 「時系列コントローラーパネル」の真ん中の時計ボタン（固定範囲の時系列ナビ）を押す．
  1. 時系列の範囲が表示されるので，範囲の右端にあるくるくる矢印ボタン（「全範囲に設定」）を押し，範囲を最初から最後までに設定する．
  1. 「時系列コントローラーパネル」の右端の再生ボタンを押し，スライダーを動かしてアニメ表示する．

# データファイルの指定

In [1]:
# 必要なデータファイルの指定

# コスト距離ファイルの指定
# コスト距離はファイル名のリストで複数指定できる
cost_files = ['arcdis2.tif']

# 道路ラスタファイルの指定
# 通れる部分が1以上，通れない部分が0のラスタファイル
road_file = 'road123.tif'

# スタート地点のgeojsonファイルの指定
start_geojson = 'start_228.geojson'

# ゴール地点のgeojsonファイルの指定
goal_geojson = 'goaldata3.geojson'



# ライブラリのインポート

Google Colaboratoryを使う場合とローカルPCで実行する場合を自動判別して
必要なライブラリをインストールする．

In [2]:
import sys

# Google Colaboratoryを使用している場合は以下の必須ライブラリをインストールする
# mesa: マルチエージェントシミュレーションライブラリ
# geopandas: 地理情報データを扱うライブラリ
if 'google.colab' in sys.modules:
  %pip install mesa==2.4.0
  %pip install geopandas
  %matplotlib notebook
  # google.coalb: GoogleDrive内のファイルにアクセスするために必要
  from google.colab import drive

# 必要なライブラリの読み込み
# numpy: 配列を扱うライブラリ
# tqdm: プログレスバーのライブラリ
# random: 乱数を扱うライブラリ
import numpy as np
from tqdm import tqdm
import random

# matplotlob: グラフ表示を行うライブラリ
# matplotlib..cm: matplotlibでカラーマップを扱うライブラリ
# matplotlib animation: アニメーション作成ライブラリ
# Ipython.display: アニメーション表示用ライブラリ
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import animation, rc
from matplotlib.animation import ArtistAnimation
#from IPython.display import HTML

# datetime: 時刻日付型のライブラリ
from datetime import date, time, datetime, timedelta

# mesa: マルチエージェントシミュレーションを行うライブラリ
from mesa import Agent, Model

# MultiGrid: 同一グリッドに複数エージェントを配置できるスペース
# SingleGrid: 同一グリッドに単一エージェントのみを配置できるスペース
from mesa.space import MultiGrid, SingleGrid

# RandomActivation: エージェントを動かす順番をランダムに決定
# SimultaneousActivation: エージェントを同時に動かす
from mesa.time import RandomActivation, SimultaneousActivation

# DataCollector: データコレクター
from mesa.datacollection import DataCollector

# osgeo: GeoTiffを読み込むために必要
from osgeo import gdal, gdalconst, gdal_array, osr

# geopandas: GeojJSONの読み込みとエージェントの座標計算に必要
import geopandas as gpd


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.9/268.9 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.0 MB/s eta 0:00:00


# データ用フォルダのマウント

Google Colaboratryで実行する場合とローカルPCで実行する場合を自動判別し，
それぞれのデータフォルダをマウントする．

In [3]:
# Google Colaboratoryを使う場合はGoogle Driveをマウントする
if 'google.colab' in sys.modules:
  drive.mount('/content/drive')
  # GoogleDrive内の"マイドライブ/gisday2023/"フォルダにアクセスできるような設定を行う．
  # フォルダ名の最後に`/`を必ず追加すること．
  base_dir = "/content/drive/My Drive/data/"
else:
  # ローカルPCを使う場合は./data/フォルダをマウントする
  # フォルダ名の最後に`/`を必ず追加すること．
  base_dir = "./data/"

Mounted at /content/drive


# 加重コスト距離・壁・スタート地点・ゴール地点の読み込み

In [4]:
# 加重コスト距離の読み込み
# cost_filesで指定されたGeoTIFFファイルを順次読み込み，リストcostに格納する．
cost = list()
for i in range(len(cost_files)):
  cost_file = gdal.Open(base_dir + cost_files[i])
  cost.append(np.where(cost_file.ReadAsArray() > 0, cost_file.ReadAsArray(), 0))

# GeoTIFF画像の左上座標（ULX, ULY），ピクセルサイズ（Ysize, Xsize）を取得する．
ULX, Xsize, dum1, ULY, dum2, Ysize = cost_file.GetGeoTransform()

# 道路（壁）の読み込み
wall_file = gdal.Open(base_dir + road_file)
wall = np.where(wall_file.ReadAsArray() >= 1, 0, 1)

# Load the new speed data from GeoTIFF
# 速度情報を持つラスタファイルの指定
speed_file = 'idw1.tif' # ここを実際の速度ラスタファイル名に変更してください
speed_dataset = gdal.Open(base_dir + speed_file)
speed_data = speed_dataset.ReadAsArray()

# NoDataを取得
nodata_val = speed_dataset.GetRasterBand(1).GetNoDataValue()

bins = [0.32, 0.38]

# numpy.digitize で分類
# speed_data < 0.32       -> index 0 (+1でカテゴリ1)
# 0.32 <= speed_data < 0.38 -> index 1 (+1でカテゴリ2)
# 0.38 <= speed_data      -> index 2 (+1でカテゴリ3)
classified_data = np.digitize(speed_data, bins, right=False) + 1



# 値とコメントを一致させたマッピング
speed_map = {
    1: 1.0, # カテゴリ1 (0.32未満): 減速なし
    2: 0.7, # カテゴリ2 (0.32-0.38): 30%減速
    3: 0.5  # カテゴリ3 (0.38以上):  50%減速 (※必要に応じて追加)
}



# スタート地点のgeojsonの読み込み
# ラスタデータの周辺部にあるスタート地点は範囲外になる可能性があるために削除する
start_location = gpd.read_file(base_dir + start_geojson).explode(index_parts = True)\
  .cx[ULX+Xsize: ULX + ((cost_file.RasterXSize - 1) * Xsize), ULY + ((cost_file.RasterYSize + 1) * Ysize): ULY + Ysize]

# ゴール地点のgeojsonの読み込み
goal_location = gpd.read_file(base_dir + goal_geojson).explode(index_parts = True)

# ゴール地点の座標を取得し、グリッド範囲内にクリップする
grid_width = wall.shape[1]
grid_height = wall.shape[0]

goal_x = ((goal_location.geometry.x - ULX) // Xsize).astype(int)
goal_y = ((goal_location.geometry.y - ULY) // Ysize).astype(int)

print("=== シミュレーション空間の範囲確認 ===")

# 1. グリッド（マス目）としての範囲
# エージェントが動けるのは x=0 から width-1 までです
h, w = wall.shape
print(f"【グリッドサイズ】")
print(f"幅 (Width) : {w} マス")
print(f"高さ(Height): {h} マス")
print(f"有効なX座標: 0 〜 {w - 1}")
print(f"有効なY座標: 0 〜 {h - 1}")

# 2. 地図上の座標範囲（メートル等）
# GISデータとしての端から端までの座標です
min_geo_x = ULX
max_geo_x = ULX + (w * Xsize)
max_geo_y = ULY
min_geo_y = ULY + (h * Ysize) # Ysizeは通常マイナスの値(北が上)なので足すと南端になる

print(f"\n【地図座標の範囲】")
print(f"X座標 (東経など): {min_geo_x:.2f} 〜 {max_geo_x:.2f}")
print(f"Y座標 (北緯など): {min_geo_y:.2f} 〜 {max_geo_y:.2f}")

# 3. データの整合性チェック
# 壁データと降灰データのサイズが合っているか確認
print(f"\n【データ整合性チェック】")
if wall.shape == classified_data.shape:
    print("✅ OK: 道路データと速度データのサイズは一致しています。")
else:
    print(f"⚠️ 警告: サイズが不一致です！")
    print(f"  道路(wall): {wall.shape}")
    print(f"  速度(classified): {classified_data.shape}")

# ファイルが読めているか・件数・CRSを確認
raw = gpd.read_file(base_dir + start_geojson)
print(f"件数: {len(raw)}")
print(f"CRS: {raw.crs}")
print(f"X範囲: {raw.geometry.x.min():.2f} 〜 {raw.geometry.x.max():.2f}")
print(f"Y範囲: {raw.geometry.y.min():.2f} 〜 {raw.geometry.y.max():.2f}")

# ラスタのフィルタ範囲と比較
print(f"\nラスタX範囲: {ULX:.2f} 〜 {ULX + cost_file.RasterXSize * Xsize:.2f}")
print(f"ラスタY範囲: {ULY + cost_file.RasterYSize * Ysize:.2f} 〜 {ULY:.2f}")


/usr/local/lib/python3.12/dist-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


=== シミュレーション空間の範囲確認 ===
【グリッドサイズ】
幅 (Width) : 400 マス
高さ(Height): 393 マス
有効なX座標: 0 〜 399
有効なY座標: 0 〜 392

【地図座標の範囲】
X座標 (東経など): 10037.57 〜 13178.53
Y座標 (北緯など): -8094.85 〜 -5008.86

【データ整合性チェック】
⚠️ 警告: サイズが不一致です！
  道路(wall): (393, 400)
  速度(classified): (393, 401)
件数: 1050
CRS: EPSG:6670
X範囲: 10037.57 〜 11673.44
Y範囲: -6971.96 〜 -5008.86

ラスタX範囲: 10037.57 〜 12039.93
ラスタY範囲: -6971.96 〜 -5008.86


# スペースとエージェントを定義する

In [5]:
import random
import numpy as np
import geopandas as gpd
from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

# ---------------------------------------------------------
# 1. モデルクラス: EvacModel
# ---------------------------------------------------------
class EvacModel(Model):
    def __init__(self, width, height, start_loc, goal_loc, wall, cost_data, speed_map, classified_data, ULX, ULY, Xsize, Ysize):

        # --- 基本設定 ---
        self.grid = MultiGrid(width, height, torus=False)
        self.wall = wall
        self.speed_data = classified_data
        self.speed_map = speed_map
        self.evac_comp = 0
        self.schedule = RandomActivation(self)
        self.cost = cost_data

        # 描画用リスト
        self.wall_x = []
        self.wall_y = []
        self.goal_x = []
        self.goal_y = []

        # データの空チェック
        if start_loc is None:
            print("!!! FATAL ERROR: start_loc is None !!!")
        elif len(start_loc) == 0:
            print("!!! FATAL ERROR: start_loc is EMPTY (0 rows) !!!")
        else:
            print(f"4. Input Data: {len(start_loc)} rows found.")
            if 'setaiID' in start_loc.columns:
                print(f"   - Grouping by 'setaiID'... (Unique families: {start_loc['setaiID'].nunique()})")
            else:
                print(f"!!! ERROR: 'setaiID' column missing. Available: {list(start_loc.columns)}")

        grouped_data = start_loc.groupby('setaiID')
        agent_id_counter = 0

        # ループが回ったかどうかのフラグ
        loop_entered = False
        skipped_count = 0

        # 世帯ごとにループ処理
        for setaiID, group in grouped_data:
            loop_entered = True

            # 世帯ルール
            family_size = len(group)
            target_evac_no = group.iloc[0]['cost_ID']

            # 時間設定（世帯で統一）
            start_delay_map = {1: 0, 2: 5, 3: 10, 4: 15}
            size_penalty = start_delay_map.get(family_size, 15)
            base_reaction = random.gauss(15, 10)
            calc_start_move = int(max(0, base_reaction + size_penalty))

            # メンバー個別の生成ループ
            for i, (index, row) in enumerate(group.iterrows()):

                # 座標変換 (GIS -> Grid)
                raw_x = row.geometry.x
                raw_y = row.geometry.y

                calc_x = (raw_x - ULX) // Xsize
                calc_y = (raw_y - ULY) // Ysize

                x = int(calc_x)
                y = int(calc_y)

                # 範囲外エラーチェックと報告
                is_in_bounds = (0 <= x < width and 0 <= y < height)

                if not is_in_bounds:
                    skipped_count += 1
                    if skipped_count <= 5:
                        print(f"[SKIP] Agent {agent_id_counter} (Fam {setaiID})")
                        print(f"   - GIS: ({raw_x:.2f}, {raw_y:.2f})")
                        print(f"   - Grid Calc: ({x}, {y})")
                        if not (0 <= x < width): print(f"   -> X is BAD. Range: 0 ~ {width}")
                        if not (0 <= y < height): print(f"   -> Y is BAD. Range: 0 ~ {height}")
                    continue

                # 全員を独立したエージェントとして生成
                agent = EvacAgent(agent_id_counter, self, setaiID, family_size)

                # 共通属性セット
                agent.evac_no = target_evac_no
                agent.start_move = calc_start_move
                agent.pop = 1

                # 配置実行
                self.schedule.add(agent)
                self.grid.place_agent(agent, (x, y))

                agent_id_counter += 1

        # データコレクター設定
        # reached_goal_id を追加：どのゴールに到達したかを記録
        self.datacollector = DataCollector(
            model_reporters={"Goal": compute_goal, "evacuator": goal_pop, "pop_sum": pop_sum},
            agent_reporters={"Pos": "pos", "Goal": "mygoal", "Behavior": "behavior", "speed": "speed", "reached_goal_id": "reached_goal_id"}
        )
        self.datacollector.collect(self)

        # 壁エージェント配置
        wall_y_indices, wall_x_indices = np.where(self.wall > 0)
        for i in range(len(wall_x_indices)):
            wx, wy = wall_x_indices[i], wall_y_indices[i]
            a = WallAgent(agent_id_counter, self)
            self.grid.place_agent(a, (int(wx), int(wy)))
            self.wall_x.append(int(wx))
            self.wall_y.append(int(wy))
            agent_id_counter += 1

        # ゴールエージェント配置
        # GOAL_IDカラムをgeojsonから取得して使用する
        goal_x_list = ((goal_loc.geometry.x - ULX) // Xsize).to_list()
        goal_y_list = ((goal_loc.geometry.y - ULY) // Ysize).to_list()
        goal_id_list = goal_loc['GOAL_ID'].to_list()
        for i in range(len(goal_x_list)):
            gx, gy = goal_x_list[i], goal_y_list[i]
            if 0 <= gx < width and 0 <= gy < height:
                a = GoalAgent(agent_id_counter, self, goal_id=goal_id_list[i])
                self.grid.place_agent(a, (int(gx), int(gy)))
                self.goal_x.append(int(gx))
                self.goal_y.append(int(gy))
                agent_id_counter += 1

    # メソッド定義
    def step(self):
        self.evac_comp = 0
        self.schedule.step()
        self.datacollector.collect(self)

    def calc_dist(self, loc1, loc2):
        return np.sqrt(np.sum(np.square(np.array(loc1) - np.array(loc2))))

# ---------------------------------------------------------
# 2. 外部関数
# ---------------------------------------------------------
def goal_pop(model):
    return model.evac_comp

def compute_goal(model):
    agents = [agent for agent in model.schedule.agents if isinstance(agent, EvacAgent)]
    if not agents: return 0
    agent_goal = sum([agent.mygoal for agent in agents])
    return agent_goal

def pop_sum(model):
    agents = [agent for agent in model.schedule.agents if isinstance(agent, EvacAgent)]
    if not agents: return 0
    return sum([agent.pop for agent in agents if agent.mygoal == 1])

# ---------------------------------------------------------
# 3. エージェントクラス群
# ---------------------------------------------------------
class EvacAgent(Agent):
    def __init__(self, unique_id, model, setaiID, family_size):
        super().__init__(unique_id, model)
        self.mygoal = 0
        self.wealth = 0
        self.category = 'evacagent'
        self.speed_remainder = 0.0

        self.setaiID = setaiID
        self.pop = 1

        # 速度決定ロジック (世帯人数のみによる傾斜)
        speed_by_size_map = {1: 1.0, 2: 0.9, 3: 0.8, 4: 0.7}
        self.base_speed = speed_by_size_map.get(family_size, 0.7)

        self.speed = self.base_speed
        self.strategy = 100
        self.behavior = 0
        self.evac_no = 0
        self.start_move = 0
        self.reached_goal_id = -1  # 未到達は-1、到達後はGOAL_IDを記録

    def move(self):
        possible_steps = list(self.model.grid.get_neighborhood(self.pos, moore = True, include_center = False))
        possible_steps2 = [a for a in possible_steps if (len(self.model.grid.get_cell_list_contents(a)) < 1)]
        possible_steps2.extend([a.pos for a in self.model.grid.get_neighbors(self.pos, moore = True, include_center = False) if a.category == 'goal'])
        possible_steps3 = []
        neighbor_agents = self.model.grid.get_neighbors(self.pos, moore = True, radius = self.strategy, include_center = False)

        if 'goal' in [a.category for a in neighbor_agents]:
            goal_list = [a for a in neighbor_agents if a.category == 'goal']
            goal_dists = [self.model.calc_dist(a.pos, self.pos) for a in goal_list]
            goal_nn = goal_list[goal_dists.index(min(goal_dists))]
            goalpos = [goal_nn.pos]
            self_goal_dis = self.model.calc_dist(goalpos, self.pos)
            distances = [self.model.calc_dist(goalpos, a) for a in possible_steps2]
            possible_steps3 = [a for a, b in zip(possible_steps2, distances) if b < self_goal_dis]

        if len(possible_steps3) > 0: target_position = self.random.choice(possible_steps3)
        elif len(possible_steps2) > 0: target_position = self.random.choice(possible_steps2)
        else: target_position = self.pos
        self.execute_move(target_position)

    def move_cost(self):
        possible_steps = list(self.model.grid.get_neighborhood(self.pos, moore = True, include_center = False))
        possible_steps2 = [a for a in possible_steps if (len(self.model.grid.get_cell_list_contents(a)) < 1)]
        possible_steps2.extend([a.pos for a in self.model.grid.get_neighbors(self.pos, moore = True, include_center = False) if a.category == 'goal'])

        if len(possible_steps2) == 0:
            target_position = self.pos
        else:
            try:
                costs = [self.model.cost[self.evac_no][b][a] for a, b in possible_steps2]
                min_cost_index = costs.index(min(costs))
                target_position = possible_steps2[min_cost_index]
            except IndexError as e:
                target_position = self.pos

        self.execute_move(target_position)

    def execute_move(self, target_position):
        dx = target_position[0] - self.pos[0]
        dy = target_position[1] - self.pos[1]
        dir_x, dir_y = 0, 0
        if dx != 0: dir_x = np.sign(dx)
        if dy != 0: dir_y = np.sign(dy)
        steps_to_move = int(self.speed)

        #移動ロジック
        #---------------------
        self.speed_remainder += (self.speed - steps_to_move)
        if self.speed_remainder >= 1.0:
            steps_to_move += int(self.speed_remainder)
            self.speed_remainder -= int(self.speed_remainder)

        #----------------------

        current_pos = self.pos
        for _ in range(steps_to_move):
            next_pos = (current_pos[0] + dir_x, current_pos[1] + dir_y)
            if (0 <= next_pos[0] < self.model.grid.width and 0 <= next_pos[1] < self.model.grid.height and self.model.wall[next_pos[1]][next_pos[0]] == 0 and len(self.model.grid.get_cell_list_contents(next_pos)) < 1):
                current_pos = next_pos
            else: break
        if current_pos != self.pos: self.model.grid.move_agent(self, current_pos)
        neighbor_agents = self.model.grid.get_neighbors(self.pos, moore = True, radius = 2, include_center = False)
        if 0 in [a.behavior for a in neighbor_agents if a.category == 'evacagent']: self.behavior = 0

    def if_goal(self):
        # 隣接セルにいるゴールエージェントを取得
        neighbors = self.model.grid.get_neighbors(self.pos, moore=True, include_center=True)
        goal_agents = [a for a in neighbors if a.category == 'goal']
        if goal_agents:
            self.mygoal = 1
            self.reached_goal_id = goal_agents[0].goal_id  # どのゴールに到達したか記録
            self.model.evac_comp += 1
            self.model.grid.remove_agent(self)

    def step(self):
        if self.start_move > 0:
            self.start_move -= 1
            return
        if self.mygoal == 1:
            return

        current_base_speed = self.base_speed
        slowdown_factor = 1.0
        x, y = self.pos
        if 0 <= y < self.model.speed_data.shape[0] and 0 <= x < self.model.speed_data.shape[1]:
            raster_value = self.model.speed_data[y][x]
            slowdown_factor = self.model.speed_map.get(raster_value, 1.0)

        # 二重傾斜（世帯人数 × 降灰）
        self.speed = current_base_speed * slowdown_factor

        if self.behavior == 1:
            self.move()
        else:
            self.move_cost()

        self.if_goal()

class GoalAgent(Agent):
    def __init__(self, unique_id, model, goal_id):
        super().__init__(unique_id, model)
        self.category = 'goal'
        self.goal_id = goal_id  # geojsonのGOAL_IDカラムの値を保持

class WallAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.category = 'wall'


# モデルの実行

In [6]:
# 1. モデルのインスタンス化（実体化）
# グリッドの幅と高さをwallデータから取得
h, w = wall.shape

# モデルにすべての変数を渡して生成する
model = EvacModel(
    width=w,
    height=h,
    start_loc=start_location,
    goal_loc=goal_location,
    wall=wall,
    cost_data=cost,
    speed_map=speed_map,
    classified_data=classified_data,
    ULX=ULX,
    ULY=ULY,
    Xsize=Xsize,
    Ysize=Ysize
)

# 2. 実行設定
runstep = 500
from tqdm import tqdm

# 3. ループ実行
print("シミュレーション開始...")
for i in tqdm(range(runstep)):
    model.step()

# 4. 結果の確認
results = model.datacollector.get_model_vars_dataframe()
print("\n--- 結果プレビュー ---")
print(results.tail())

agent_results = model.datacollector.get_agent_vars_dataframe()
print("\n--- エージェントの動きプレビュー ---")
print(agent_results.tail())

4. Input Data: 1044 rows found.
   - Grouping by 'setaiID'... (Unique families: 4)
シミュレーション開始...


/usr/local/lib/python3.12/dist-packages/mesa/agent.py:52: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  self.model.register_agent(self)
100%|██████████| 500/500 [00:17<00:00, 28.51it/s]



--- 結果プレビュー ---
     Goal  evacuator  pop_sum
496   755          2      755
497   755          0      755
498   757          2      757
499   758          1      758
500   762          4      762

--- エージェントの動きプレビュー ---
                    Pos  Goal  Behavior  speed  reached_goal_id
Step AgentID                                                   
500  455           None     1         0   0.70                2
     276           None     1         0   0.49                3
     802           None     1         0   0.70                2
     730           None     1         0   0.70                2
     704      (149, 61)     0         0   0.70               -1


# 計算結果をQGISやArcGISで直接読み込めるGeoPackage型式で保存する

In [ ]:
# ↓ この1行を先頭に追加
outfilename = 'result_301_3'  # 好きな名前に変更してOK

# 避難者エージェントの実座標を計算しGeoDataFrameにする
# データコレクターからすべてのステップにおける避難者エージェントの位置（Pos）を取得する
agent_loc = model.datacollector.get_agent_vars_dataframe()['Pos'].map(lambda x: (0, 0) if (x is None or (isinstance(x, float) and np.isnan(x))) else x)
# agent_loc = model.datacollector.get_agent_vars_dataframe()['Pos']
# 取得したPosをxとyに分解する
x, y = [a for a, b in agent_loc], [b for a, b in agent_loc]
# xとyを実座標（xcoordとycoord）に変換する
xcoord = [ULX + (x_val * Xsize) for x_val in x]
ycoord = [ULY + (y_val * Ysize) for y_val in y]
# 実座標をGeoDatqaFrameのgeometryに変換する
# CRSはstart_locationと同じものにする
agent_geometry = gpd.points_from_xy(xcoord, ycoord, crs = start_location.crs)

# データコレクターに実座標（agent_geometry）を付与する
agent_vars = gpd.GeoDataFrame(model.datacollector.get_agent_vars_dataframe(), geometry = agent_geometry)

# Object型であるPosをstrにキャストする．これをやらないとgpkgで保存できない
agent_vars[['Pos']] = agent_vars[['Pos']].astype('str')

# タイムステップを取得
stp = agent_vars.reset_index()["Step"]
# NaN値を処理：timedelta計算前に0に置換し、整数型にする
stp_filled = stp.fillna(0).astype(int)

# 基準時刻（2020/1/1 0:0:0）にタイムステップ*5秒を掛けたものを加えてDateTimeにする
agent_vars["Time"] = [(datetime(2020, 1, 1, 0, 0, 0) + timedelta(seconds = (a *5)))  for a in stp_filled]

# データコレクターなどをファイルに保存する
agent_vars.reset_index().to_file(base_dir + outfilename + '_agent_vars.gpkg', layer = 'agent_vars', driver = 'GPKG')
#agent_vars.reset_index().to_csv(base_dir + outfilename + '_agent_vars.csv')

# エージェントデータをCSVでも保存
agent_vars.reset_index().drop(columns='geometry').to_csv(base_dir + outfilename + '_agent_vars.csv', index=False)


#避難所ごとのゴール人数の集計

In [10]:
import pandas as pd

df = pd.read_csv(base_dir + 'result_301_3_agent_vars.csv')

total_agents = 4000  # 全エージェント数

# --- 最終ステップ（500）の集計 ---
df_last = df[df['Step'] == 500]

summary = df_last.groupby('reached_goal_id')['AgentID'].count().reset_index()
summary.columns = ['reached_goal_id', '到達人数（圧縮）']

goal_summary = summary[summary['reached_goal_id'] != -1].copy()
goal_summary['実到達人数'] = goal_summary['到達人数（圧縮）'] * 4

capacity = {1: 200, 2: 400, 3: 350}
goal_summary['収容限界'] = goal_summary['reached_goal_id'].map(capacity)
goal_summary['収容率(%)'] = (goal_summary['実到達人数'] / goal_summary['収容限界'] * 100).round(1)
goal_summary['判定'] = goal_summary['収容率(%)'].apply(lambda x: '⚠️ 超過' if x > 100 else '✅ 余裕あり')

print("=== 避難所ごとの集計（最終ステップ）===")
print(goal_summary.to_string(index=False))

# --- 全体の避難完了率（最終ステップ） ---
completed_final = goal_summary['到達人数（圧縮）'].sum()  # 圧縮人数での完了者
completed_final_real = completed_final * 4               # 実人数換算
rate_final = completed_final_real / total_agents * 100

print(f"\n=== 全体の避難完了率（最終ステップ500） ===")
print(f"  完了人数: {completed_final_real}人 / {total_agents}人")
print(f"  避難完了率: {rate_final:.1f}%")

# --- 360ステップ時点の途中経過 ---
df_360 = df[df['Step'] == 360]

summary_360 = df_360[df_360['reached_goal_id'] != -1].groupby('reached_goal_id')['AgentID'].count().reset_index()
summary_360.columns = ['reached_goal_id', '到達人数（圧縮）']
summary_360['実到達人数'] = summary_360['到達人数（圧縮）'] * 4

completed_360 = summary_360['到達人数（圧縮）'].sum()
completed_360_real = completed_360 * 4
rate_360 = completed_360_real / total_agents * 100

print(f"\n=== 全体の避難完了率（途中経過：360ステップ）===")
print(f"  完了人数: {completed_360_real}人 / {total_agents}人")
print(f"  避難完了率: {rate_360:.1f}%")

=== 避難所ごとの集計（最終ステップ）===
 reached_goal_id  到達人数（圧縮）  実到達人数  収容限界  収容率(%)    判定
               1       116    464   200   232.0 ⚠️ 超過
               2       323   1292   400   323.0 ⚠️ 超過
               3       309   1236   350   353.1 ⚠️ 超過

=== 全体の避難完了率（最終ステップ500） ===
  完了人数: 2992人 / 4000人
  避難完了率: 74.8%

=== 全体の避難完了率（途中経過：360ステップ）===
  完了人数: 2288人 / 4000人
  避難完了率: 57.2%


# アニメーション作成（エージェントや計算ステップ数が多いとかなり時間がかかる）

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import ArtistAnimation
from IPython.display import HTML

def create_simulation_animation_artist(model, run_steps):
    # エージェントとモデルのデータを取得
    agent_data = model.datacollector.get_agent_vars_dataframe().reset_index()
    model_data = model.datacollector.get_model_vars_dataframe()

    fig, ax = plt.subplots(figsize=(10, 8))

    # 1. 背景の描画 (サイズ違いを自動補正)
    min_h = min(model.wall.shape[0], model.speed_data.shape[0])
    min_w = min(model.wall.shape[1], model.speed_data.shape[1])

    wall_clipped = model.wall[:min_h, :min_w]
    speed_clipped = model.speed_data[:min_h, :min_w]

    background = np.where(wall_clipped > 0, -1, speed_clipped)
    ax.imshow(background, cmap='viridis', origin='upper', alpha=0.3)

    ax.set_title("阿蘇山噴火時 避難シミュレーション プロトタイプ")
    ax.set_xlabel("Grid X")
    ax.set_ylabel("Grid Y")

    frames = [] # ここに各ステップの描画結果（Artist）を格納していきます

    # 2. パラパラ漫画の各ページ（フレーム）を作成
    for step in range(run_steps):
        step_data = agent_data[agent_data['Step'] == step]
        pos_list = [p for p in step_data['Pos'] if isinstance(p, tuple)]

        # エージェントのプロット
        if pos_list:
            x, y = zip(*pos_list)
            scatter = ax.scatter(x, y, s=10, c='blue', edgecolors='white', linewidths=0.5)
        else:
            scatter = ax.scatter([], [], s=10, c='blue', edgecolors='white', linewidths=0.5)

        # テキスト情報のプロット
        evacuated = model_data.iloc[step]['evacuator']
        stats_text = ax.text(0.02, 0.95, f"Step: {step}\n避難完了数: {int(evacuated)}人",
                             transform=ax.transAxes, bbox=dict(facecolor='white', alpha=0.8))

        # このフレームで描画した要素（点と文字）をリストに追加
        frames.append([scatter, stats_text])

    # 3. リストに貯めたフレームをアニメーション化
    ani = ArtistAnimation(fig, frames, interval=100, blit=True, repeat_delay=1000)


    return ani
# --- 実行部分 ---
print("アニメーションを生成中... (パラパラ漫画を作成しています)")
ani = create_simulation_animation_artist(model, runstep)

# to_jshtml()でデータ化してからfigureを閉じる
html = ani.to_jshtml()


HTML(html)


# Colab上で動画として再生・表示する
HTML(ani.to_jshtml())

# ゴール過程のグラフ化

In [ ]:
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111)

goal = model.datacollector.get_model_vars_dataframe()

ax.plot(goal)
plt.show